In [6]:
# Grupo 7
## Integrantes 
### Pablo Sarzosa
### Elisa Herrera
### Doménica Rodríguez 
### Andrés Albarracín

In [15]:
use strict;
use warnings;
use Data::Dump qw(dump);
use Data::Dumper qw(Dumper);
use AI::MXNet qw(mx);
use List::Util qw(max min shuffle);
use d2l;
use d2l::timer qw(time);
use d2l::animator;
use d2l::accumulator;
IPerl->load_plugin('Chart::Plotly');
use Chart::Plotly::Plot


Error: "use" not allowed in expression at reply input line 8, at end of line
syntax error at reply input line 8, near "use d2l
use d2l::timer "
BEGIN not safe after errors--compilation aborted at reply input line 9.



In [13]:
my $num_inputs = 784;
my $num_outputs = 10;

my @W = map {rand(0.01)} 1..($num_inputs * $num_outputs);
my @b = (0) x $num_outputs;

0000000000

In [4]:
my $X = pdl([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]]);

# Suma de columnas con keepdims=True
my $suma_cols = $X->sumover->dummy(0);

# Suma de filas con keepdims=True
my $suma_rows = $X->sumover(1)->dummy(1);

print $suma_cols->flat;
print $suma_rows->flat;


Error: Undefined subroutine &main::pdl called at reply input line 1.


In [5]:
sub softmax {
    my ($X) = @_;
    my $X_exp = exp($X);
    my $partition = $X_exp->sumover(1)->dummy(1);
    return $X_exp / $partition;
}

my $X = p2l([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]]);
print softmax($X);


Warning: Subroutine softmax redefined at reply input line 1.


Error: Undefined subroutine &main::p2l called at reply input line 8.


In [6]:
my $X = random(2,5)->normal(0,1);
my $X_prob = softmax($X);
print $X_prob;
print $X_prob->sumover(1);


Error: Undefined subroutine &main::random called at reply input line 1.


In [7]:
sub net {
    my ($X) = @_;
    my $X_reshape = $X->reshape(-1, $W->shape->at(0));
    my $X_dot_W = $X_reshape x $W;
    my $X_dot_W_plus_b = $X_dot_W + $b;
    return softmax($X_dot_W_plus_b);
}

In [8]:
my $y = pdl([0, 2]);
my $y_hat = pdl([[0.1, 0.3, 0.6], [0.3, 0.2, 0.5]]);
my $result = $y_hat->dice_axis(0,$y);


Error: Undefined subroutine &main::pdl called at reply input line 1.


In [9]:
use PDL::Ufunc qw/log/;

sub cross_entropy {
    my ($y_hat, $y) = @_;
    return -log($y_hat->dice_axis(0,$y));
}

cross_entropy($y_hat, $y);


Error: "log" is not exported by the PDL::Ufunc module
Can't continue after import errors at /usr/local/lib/perl5/site_perl/5.32.1/x86_64-linux/PDL/Exporter.pm line 66.
BEGIN failed--compilation aborted at reply input line 1.



In [10]:
sub accuracy {
    my ($y_hat, $y) = @;
    if (scalar(@{$y_hat->[0]}) > 1) {
        $y_hat = [map {$->[0]} @$y_hat];
    }
    my $cmp = [map {$y_hat->[$] == $y->[$]} 0..$#$y_hat];
    return scalar(grep {$_ == 1} @$cmp);
}
accuracy(y_hat, y) / scalar(@$y);

Warning: Number found where operator expected at reply input line 6, near "} 0"

	(Missing operator before  0?)


Error: Transliteration replacement not terminated at reply input line 9.



In [11]:
sub evaluate_accuracy {
    my ($net, $data_iter) = @;
    my $metric = {'correct' => 0, 'total' => 0};
    for my $data (@$data_iter) {
        my ($X, $y) = @$data;
        my $cmp = [map {$net->($X)->[$] == $y->[$]} 0..$#$y];
        $metric->{'correct'} += scalar(grep {$ == 1} @$cmp);
        $metric->{'total'} += scalar(@$y);
        }
    return $metric->{'correct'} / $metric->{'total'};
}

Warning: Number found where operator expected at reply input line 6, near "} 0"

	(Missing operator before  0?)


Error: syntax error at reply input line 3, near "@;
    my "
syntax error at reply input line 6, at EOF
BEGIN not safe after errors--compilation aborted at reply input line 15.



In [12]:
package Accumulator;

sub new {
    my ($class, $n) = @_;
    my $self = {
    data => [ (0.0) x $n ],
    };
    return bless $self, $class;
}

sub add {
    my ($self, @args) = @;
    for (0..$#{$self->{data}}) {
    $self->{data}->[$] += $args[$_];
    }
}

sub reset {
    my ($self) = @_;
    $self->{data} = [ (0.0) x scalar(@{$self->{data}}) ];
}

sub get {
    my ($self, $idx) = @_;
    return $self->{data}->[$idx];
}

1;

Warning: "my" variable $self masks earlier declaration in same statement at reply input line 20.

"my" variable $self masks earlier declaration in same scope at reply input line 24.

"my" variable @_ masks earlier declaration in same scope at reply input line 24.

"my" variable $self masks earlier declaration in same statement at reply input line 25.

"my" variable $idx masks earlier declaration in same statement at reply input line 25.


Error: syntax error at reply input line 13, near ") {"
syntax error at reply input line 18, near "}

sub reset "
syntax error at reply input line 21, near ";
}"
syntax error at reply input line 26, near ";
}"
BEGIN not safe after errors--compilation aborted at reply input line 32.



In [13]:
evaluate_accuracy($net, $test_iter);

Error: Undefined subroutine &main::evaluate_accuracy called at reply input line 1.
